In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [26]:
!pip install langchainhub
!pip install openai
!pip install python-dotenv
!pip install langchain
!pip install langchain_community
!pip install -qU langchain-openai
!pip install faiss-cpu
!pip install faiss-gpu
!pip install langchainhub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 40.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 6.8 MB/s eta 0:00:00


In [21]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv('/content/drive/Othercomputers/내 노트북/code/MarketingAI-Bot/.env')

True

In [23]:
from langchain_community.document_loaders.csv_loader import CSVLoader
import os

loader = CSVLoader(file_path="/content/drive/Othercomputers/내 노트북/code/MarketingAI-Bot/output.csv")
data = loader.load()


GPT_API_KEY = os.environ.get('GPT_API_KEY')
ORG_ID = os.environ.get('ORG_ID')

In [10]:
data[0]

Document(metadata={'source': '/content/drive/Othercomputers/내 노트북/code/MarketingAI-Bot/output.csv', 'row': 0}, page_content="Index: 1\n생성시간: 2024.07.05 08:52:57\n게시글내용: 𝙉𝙚𝙬𝙡𝙮 𝙡𝙖𝙪𝙣𝙘𝙝!어글리 러블리 마스크팩 트래블러스 키트 출시💃어글리 러블리 마스크 팩 3종 모두를파우치에 쏙, 여행 시 휴대가 간편한30ml 미니어처 사이즈로 만나보세요.사랑스러운 패키지 디자인을 적용해소중한 사람들에게, 늘 고마운 나에게선물해보시는 건 어떨까요?*마스크팩 3종 트래블러스 키트는 카카오톡 선물하기에서 만나보실 수 있습니다.\n해쉬태그: ['#어글리러블리', '#uglylovely']\nimg_path: ./images\\0.jpg")

In [64]:
from langchain_openai import OpenAIEmbeddings

# OpenAI의 "text-embedding-ada-002" 모델을 사용하여 임베딩을 생성합니다.
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002",openai_api_key=GPT_API_KEY)

In [135]:
from langchain_community.vectorstores import FAISS

# 저장
vectorstore = FAISS.from_documents(data, embeddings)

# 저장된 DB의 리트리버를 가져와서 검색에 사용한다.
# 서치 타입 mmr은 유사한 정보를 가진 문서는 제외하는 알고리즘을 사용하는것이다.
# 검색인자로 k는 반환할 문서의 수를 지정한다.
# retriever = vectorstore.as_retriever(search_type="mmr", search_kwargs={"k": 5})

# 유사도를 이용한 retiever 생성
retriever = vectorstore.as_retriever(
    # 검색 유형을 "유사도 점수 임계값"으로 설정합니다.
    search_type="similarity_score_threshold",
    # 검색 인자로 점수 임계값을 0.5로 지정합니다.
    search_kwargs={
        "score_threshold": 0.5,
        "k": 7},
)


In [128]:
# FAISS 모듈에서 인덱스를 가져옴
index = vectorstore.index

# 인덱스에 저장된 벡터의 개수를 출력
vector_count = index.ntotal
print(f"백터스토어에 저장된 데이터 개수: {vector_count}")


백터스토어에 저장된 데이터 개수: 76


In [129]:
# 검색된 컨택스트를 하나로 합쳐준다.
def format_docs(docs):
  result= []
  for doc in docs:
    page_content = doc.page_content
    result.append(page_content)
    print(page_content)
  return "\n\n".join(result)

# 문서 검색 및 포맷팅
def search_and_format(retriever,question):
    docs = retriever.invoke(question)
    context = format_docs(docs)
    return {"context": context, "question": question}

In [136]:
context_and_question = search_and_format(retriever,"유자처럼 상큼한것이 없을까?")
print("-"*100)
context_and_question

Index: 7
생성시간: 2024.06.11 07:47:31
게시글내용: 유자청을 닮은 찐~득한 제형이피부를 맑고 환하게 밀착케어 해줍니다.새콤 달콤한 유자 향으로 즐기는 향기 테라피는 덤!🍋
해쉬태그: ['#어글리러블리', '#uglylovely']
img_path: ./images\6.jpg
Index: 62
생성시간: 2023.12.04 07:52:28
게시글내용: 유기농, 무농약으로 키워 삐뚤빼뚤 개성있는 생김새로 자란 못난이 무화과에 주목!는 상품 가치가 떨어져 버려지는 못난이 무화과에서 추출한 업사이클링 원료를 사용합니다.
해쉬태그: ['#어글리러블리피그마스크', '#어글리러블리', '#uglylovely']
img_path: ./images\61.jpg
Index: 9
생성시간: 2024.06.07 05:31:50
게시글내용: 그려 나 고흥 유자여!전남 고흥 못난이 유자를 업사이클링한 원료로 만든어글리 러블리 유자 마스크가 당신의 피부를 맑고 환하게
해쉬태그: ['#어글리러블리', '#uglylovely']
img_path: ./images\8.jpg
Index: 65
생성시간: 2023.11.28 09:46:57
게시글내용: 화학비료와 농약을 사용하지 않아 자연적으로 생긴 삐뚤빼뚤 영광의 상처를 가진 못난이 유자에 주목!는 상품 가치가 떨어져 버려지는 못난이 유자에서 추출한 업사이클링 원료를 사용합니다.네이버 스토어 단독 𝟮𝟵% 𝗢𝗙𝗙👆🏻상단 프로필 링크 클릭!
해쉬태그: ['#어글리러블리유자마스크', '#어글리러블리', '#uglylovely']
img_path: ./images\64.jpg
Index: 21
생성시간: 2024.04.30 08:21:40
게시글내용: 유자청을 닮아 찐~득한 텍스처를 가진어글리 러블리 유자 마스크 팩지친 피부에 깊은 보습감과 은은한 광채감으로생기 있는 피부를 경험해보세요.
해쉬태그: ['#어글리러블리', '#uglylovely']
img_path: ./images\20.jpg
Index: 8
생성시간: 2024

{'context': "Index: 7\n생성시간: 2024.06.11 07:47:31\n게시글내용: 유자청을 닮은 찐~득한 제형이피부를 맑고 환하게 밀착케어 해줍니다.새콤 달콤한 유자 향으로 즐기는 향기 테라피는 덤!🍋\n해쉬태그: ['#어글리러블리', '#uglylovely']\nimg_path: ./images\\6.jpg\n\nIndex: 62\n생성시간: 2023.12.04 07:52:28\n게시글내용: 유기농, 무농약으로 키워 삐뚤빼뚤 개성있는 생김새로 자란 못난이 무화과에 주목!는 상품 가치가 떨어져 버려지는 못난이 무화과에서 추출한 업사이클링 원료를 사용합니다.\n해쉬태그: ['#어글리러블리피그마스크', '#어글리러블리', '#uglylovely']\nimg_path: ./images\\61.jpg\n\nIndex: 9\n생성시간: 2024.06.07 05:31:50\n게시글내용: 그려 나 고흥 유자여!전남 고흥 못난이 유자를 업사이클링한 원료로 만든어글리 러블리 유자 마스크가 당신의 피부를 맑고 환하게\n해쉬태그: ['#어글리러블리', '#uglylovely']\nimg_path: ./images\\8.jpg\n\nIndex: 65\n생성시간: 2023.11.28 09:46:57\n게시글내용: 화학비료와 농약을 사용하지 않아 자연적으로 생긴 삐뚤빼뚤 영광의 상처를 가진 못난이 유자에 주목!는 상품 가치가 떨어져 버려지는 못난이 유자에서 추출한 업사이클링 원료를 사용합니다.네이버 스토어 단독 𝟮𝟵% 𝗢𝗙𝗙👆🏻상단 프로필 링크 클릭!\n해쉬태그: ['#어글리러블리유자마스크', '#어글리러블리', '#uglylovely']\nimg_path: ./images\\64.jpg\n\nIndex: 21\n생성시간: 2024.04.30 08:21:40\n게시글내용: 유자청을 닮아 찐~득한 텍스처를 가진어글리 러블리 유자 마스크 팩지친 피부에 깊은 보습감과 은은한 광채감으로생기 있는 피부를 경험해보세요.\n해쉬태그: ['#어글리러블리', '#uglylovely

In [38]:
# 랭스미스의 허브에 들어가면 미리 정의된 다양한 프롬프트들이 있다.
# hum를 통해 해당 프롬프트 탬플릿을 사용할 수 있다.
from langchain import hub

prompt = hub.pull("rlm/rag-prompt")
prompt

ChatPromptTemplate(input_variables=['context', 'question'], metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))])

In [138]:
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough, RunnableSequence

llm = ChatOpenAI(model="gpt-3.5-turbo",openai_api_key=GPT_API_KEY)

rag_chain = (
    {"context":retriever | format_docs, "question":RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [140]:
rag_chain.invoke("현재 장마철을 이용하여 유자 마스크팩을 홍보하는 홍보글을 작성해줘")

Index: 8
생성시간: 2024.06.10 06:30:42
게시글내용: 유자C의 브라이트닝 케어🤩세린, 아르기닌을 공융혼합한독자 개발 원료*를 함유한 유자 마스크 팩으로 효과적인 각질 케어와 함께 매끈해진 피부결을 경험해보세요.(*특허 제10-2269121호)
해쉬태그: ['#어글리러블리', '#uglylovely']
img_path: ./images\7.jpg
Index: 64
생성시간: 2023.11.29 08:04:25
게시글내용: 어제보다 밝은 오늘을 꿈꾸며✨유자스팀 생기톤업 비건 온열 보습팩𝑉𝑒𝑔𝑎𝑛 𝑤𝑎𝑠ℎ-𝑜𝑓𝑓 𝑚𝑎𝑠𝑘 𝑝𝑎𝑐𝑘 𝑐𝑜𝑛𝑡𝑎𝑖𝑛𝑖𝑛𝑔 𝑒𝑥𝑡𝑟𝑎𝑐𝑡 𝑜𝑓 𝑢𝑔𝑙𝑦 𝑦𝑢𝑗𝑎, 𝑤𝑖𝑡ℎ 𝑡ℎ𝑒 𝑡ℎ𝑖𝑐𝑘 𝑡𝑒𝑥𝑡𝑢𝑟𝑒 𝑓𝑒𝑒𝑙𝑖𝑛𝑔 𝑡ℎ𝑎𝑡 𝑖𝑠 𝑎𝑙𝑚𝑜𝑠𝑡 𝑙𝑖𝑘𝑒 𝑦𝑢𝑗𝑎 𝑠𝑦𝑟𝑢𝑝 𝑡𝑜 𝑐𝑟𝑒𝑎𝑡𝑒 𝑏𝑟𝑖𝑔ℎ𝑡 𝑎𝑛𝑑 ℎ𝑒𝑎𝑙𝑡ℎ𝑦 𝑔𝑙𝑜𝑤𝑖𝑛𝑔 𝑠𝑘𝑖𝑛.어글리 러블리 유자 마스크는고흥산 못난이 유자로 만든 비건 워시오프 마스크팩으로,피부 컨디션 회복에 도움을 주는 유효 성분을 함유하여 더욱 더 환하고 맑게 빛나는 피부 케어를 도와줍니다.네이버 스토어 단독 𝟮𝟵% 𝗢𝗙𝗙👆🏻상단 프로필 링크 클릭!
해쉬태그: ['#어글리러블리유자마스크', '#어글리러블리', '#uglylovely']
img_path: ./images\63.jpg
Index: 19
생성시간: 2024.05.07 08:22:24
게시글내용: 기어, 나 제주 당근 마씸제주 구좌읍 못난이 당근을 업사이클링한 원료로 만든어글리 러블리 캐롯 마스크가 당신이 잠든 사이 보습열일
해쉬태그: ['#어글리러블리', '#uglylovely']
img_path: ./images\18.jpg
Index: 21
생성시간: 2024.04.30 08:21:40
게시글내용: 유자청을 닮아 찐~득한 텍스처를 가진어글리 러블리 유자 마스크 팩지친 피부에 깊은 보습감과 은은한 광채감으로생기 있는 피부를 경험해보세요.
해쉬태그: ['#어글리러블리', '#uglylovely']
img_path: ./

'유자 마스크팩으로 각질 케어와 매끈한 피부결을 경험할 수 있습니다. 고흥산 못난이 유자로 만든 어글리 러블리 유자 마스크는 피부 컨디션 회복에 도움을 줍니다. 외출 전에는 우산을 챙기고, 집에서는 어글리 러블리 마스크 팩을 사용하여 스페셜 케어를 즐기세요.'